# Unit 7. Putting it all together (speech-to-speech translation)
- Using V100 GPU

## Speech-to-speech translation

### Speech translation

In [ ]:
# Temporary fix to install sentencepiece
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install sentencepiece

# Restart Runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


In [ ]:
# Load Whisper tiny (39M params) for this demo
import torch
from transformers import pipeline

# Use GPU if available
device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline("automatic-speech-recognition",
                model="openai/whisper-tiny",
                device=device
)

In [ ]:
# Load an italian audio sample from VoxPopuli
from datasets import load_dataset

dataset = load_dataset("facebook/voxpopuli", "it",
                       split="validation", streaming=True)
sample = next(iter(dataset))

In [ ]:
# (Optional) Listen to the sample
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
# Define a function to translate an audio, in English by default
def translate(audio):
    outputs = pipe(audio, max_new_tokens=256,
                   generate_kwargs={"task": "translate"})
    return outputs["text"]


# Apply to the sample
translate(sample["audio"].copy())

' The social psychology, the fact that it is a very important step in the space of the juridical space of the liberalisation and the protection of the rights of the'

In [ ]:
# Compare to the source
sample["raw_text"]

'Penso che questo sia un passo in avanti importante nella costruzione di uno spazio giuridico di libertà di circolazione e di protezione dei diritti per le persone in Europa.'

### Text-to-speech

In [ ]:
# Load Processor, Model and Vocoder from a pre-trained checkpoint
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [ ]:
# Place the model and vocoder on the previously defined "device" (GPU/CPU)
model.to(device)
vocoder.to(device)

SpeechT5HifiGan(
  (conv_pre): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
  (upsampler): ModuleList(
    (0): ConvTranspose1d(512, 256, kernel_size=(8,), stride=(4,), padding=(2,))
    (1): ConvTranspose1d(256, 128, kernel_size=(8,), stride=(4,), padding=(2,))
    (2): ConvTranspose1d(128, 64, kernel_size=(8,), stride=(4,), padding=(2,))
    (3): ConvTranspose1d(64, 32, kernel_size=(8,), stride=(4,), padding=(2,))
  )
  (resblocks): ModuleList(
    (0): HifiGanResidualBlock(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
        (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
      )
      (convs2): ModuleList(
        (0-2): 3 x Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      )
    )
    (1): HifiGanResidualBlock(
      (convs1): ModuleList(
        (0): Conv1d(256, 256,

In [ ]:
# Load the speaker embeddings
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors",
                                  split="validation")
speaker_embeddings = torch.tensor(
    embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
# Define function that takes text prompt as input and generates speech
def synthesise(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(inputs["input_ids"].to(device),
                                   speaker_embeddings.to(device),
                                   vocoder=vocoder
                                   )
    return speech.cpu() # to play it back in the notebook


# Check that it works
speech = synthesise("Hey there! This is a test! How is it working?")

Audio(speech, rate=16000)

### Creating a STST (Speech-To-Speech Translation) demo

In [ ]:
# Sanity check to make sure the two models (Speech translation and
# Text-to-speech) work together
import numpy as np

target_dtype = np.int16 # format expected by Gradio
max_range = np.iinfo(target_dtype).max


def speech_to_speech_translation(audio):
    translated_text = translate(audio)
    synthesised_speech = synthesise(translated_text)
    synthesised_speech = (
        synthesised_speech.numpy() * max_range).astype(target_dtype)
    return 16000, synthesised_speech


# Check that it gives the expected result
sampling_rate, synthesised_speech = speech_to_speech_translation(
    sample["audio"])

Audio(synthesised_speech, rate=sampling_rate) # issue with the end of the speech

In [ ]:
# Temporary fix to install Gradio
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 11.8 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579

In [ ]:
# Build the Gradio demo
import gradio as gr

demo = gr.Blocks()

mic_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy")
    )

file_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy")
)

with demo:
    gr.TabbedInterface(
        [mic_translate, file_translate], ["Microphone", "Audio File"])

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:950: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


## Creating a voice assistant

### Wake word detection

In [ ]:
# Load pre-trained model's checkpoint
from transformers import pipeline
import torch

# Use GPU if available
device = "cuda:0" if torch.cuda.is_available() else "cpu"

classifier = pipeline("audio-classification",
                      model="MIT/ast-finetuned-speech-commands-v2",
                      device=device
)

In [ ]:
# Check the labels the model was trained on
classifier.model.config.id2label

{0: 'backward',
 1: 'follow',
 2: 'five',
 3: 'bed',
 4: 'zero',
 5: 'on',
 6: 'learn',
 7: 'two',
 8: 'house',
 9: 'tree',
 10: 'dog',
 11: 'stop',
 12: 'seven',
 13: 'eight',
 14: 'down',
 15: 'six',
 16: 'forward',
 17: 'cat',
 18: 'right',
 19: 'visual',
 20: 'four',
 21: 'wow',
 22: 'no',
 23: 'nine',
 24: 'off',
 25: 'three',
 26: 'left',
 27: 'marvin',
 28: 'yes',
 29: 'up',
 30: 'sheila',
 31: 'happy',
 32: 'bird',
 33: 'go',
 34: 'one'}

In [ ]:
# Select the keyword that will be used
# I'll go with "sheila"
classifier.model.config.id2label[30]

'sheila'

In [ ]:
# Define a trigger function
# ffmpeg_microphone_live continually listens to the microphone and passes
# the audio to the classification model for inference
from transformers.pipelines.audio_utils import ffmpeg_microphone_live


def launch_fn(wake_word="sheila",
              prob_threshold=0.6,
              chunk_length_s=2.0, # length of chuncks of audio
              stream_chunk_s=0.25, # minimal tempprary audio length
              debug=False
              ):
  if wake_word not in classifier.model.config.label2id.keys():
     raise ValueError(
        f"Wake word {wake_word} not in set of valid class labels, pick a wake word in the set {classifier.model.config.label2id.keys()}."
        )

  sampling_rate = classifier.feature_extractor.sampling_rate

  mic = ffmpeg_microphone_live(
      sampling_rate=sampling_rate,
      chunk_length_s=chunk_length_s,
      stream_chunk_s=stream_chunk_s
      )

  print("Listening for wake word...")
  for prediction in classifier(mic):
    prediction = prediction[0] # class label with highest probability
    if debug:
      print(prediction)
    if prediction["label"] == wake_word:
      if prediction["score"] > prob_threshold:
        return True

In [ ]:
# DO NOT RUN: keeps listening with no output

# Run the function
launch_fn(debug=True)

### Speech transcription

In [ ]:
# Load the model's checkpoint (Whisper tiny)
transcriber = pipeline("automatic-speech-recognition",
                       model="openai/whisper-tiny.en",
                       device=device
)

In [ ]:
# Define a function to transcribe the recording
import sys


def transcribe(chunk_length_s=5.0, stream_chunk_s=1.0):
    sampling_rate = transcriber.feature_extractor.sampling_rate

    mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )

    print("Start speaking...")
    for item in transcriber(mic, generate_kwargs={"max_new_tokens": 128}):
        sys.stdout.write("\033[K")
        print(item["text"], end="\r")
        if not item["partial"][0]:
            break

    return item["text"] # looks like this needs a fix

In [ ]:
# DO NOT RUN: UnboundLocalError: local variable 'spoken_' referenced before assignment

# Then run it
transcribe()

### Language model query

In [ ]:
# Link notebook to the hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Define the LLM to query, hosted on Hugging Face Hub
# https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
from huggingface_hub import HfFolder
import requests


def query(text, model_id="tiiuae/falcon-7b-instruct"):
    api_url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {HfFolder().get_token()}"}
    payload = {"inputs": text}

    print(f"Querying...: {text}")
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()[0]["generated_text"][len(text) + 1 :]

In [ ]:
# Run it
query("What does Hugging Face do?")

Querying...: What does Hugging Face do?


'Hugging Face is a company that provides natural language processing and machine learning tools for developers. They'

### Synthesize text

In [ ]:
# Select model for TTS
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)

In [ ]:
# (Re)load the speaker embeddings
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors",
                                  split="validation")
speaker_embeddings = torch.tensor(
    embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [ ]:
# Define function that takes text prompt as input and generates speech
def synthesise(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(inputs["input_ids"].to(device),
                                   speaker_embeddings.to(device),
                                   vocoder=vocoder
    )
    return speech.cpu()


# Ensure that it works properly
from IPython.display import Audio

audio = synthesise(
    "Hugging Face is a company that provides natural language processing and machine learning tools for developers."
)

Audio(audio, rate=16000)

In [ ]:
# NOT RUN HERE, launch_fn and transcribe need a fix before

# Run the assistant
launch_fn()
transcription = transcribe()
response = query(transcription)
audio = synthesise(response)

Audio(audio, rate=16000, autoplay=True)

### Generalize

In [ ]:
# Make use of the available agents
# For example, Bigcode Starcoder, free, using StableDiffusion
from transformers import HfAgent

agent = HfAgent(
    url_endpoint="https://api-inference.huggingface.co/models/bigcode/starcoder"
)

In [ ]:
# Run it
agent.run("Generate an image of a cat")

In [ ]:
# Use it with the previously defined functions
launch_fn()
transcription = transcribe()
agent.run(transcription)

## Transcribe a meeting

### Speaker diarization

In [ ]:
!pip install --upgrade pyannote.audio

# Restart Runtime

In [ ]:
# Link notebook to the hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Load the pre-trained model
from pyannote.audio import Pipeline

diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization@2.1", use_auth_token=True
)

In [ ]:
# Load a sample
from datasets import load_dataset

concatenated_librispeech = load_dataset(
    "sanchit-gandhi/concatenated_librispeech", split="train", streaming=True
)
sample = next(iter(concatenated_librispeech))

In [ ]:
# (Optional) Listen to it
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
# Pass it to the model
import torch

# Reshape as expected by the model: (channels, seq_length)
input_tensor = torch.from_numpy(sample["audio"]["array"][None, :]).float()
outputs = diarization_pipeline(
    {"waveform": input_tensor, "sample_rate": sample["audio"]["sampling_rate"]}
)

outputs.for_json()["content"]

### Speech transcription

In [ ]:
# Load the model, here Whisper base
from transformers import pipeline

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base"
)

In [ ]:
# Get the transcription, and return the timestamps
asr_pipeline(
    sample["audio"].copy(),
    generate_kwargs={"max_new_tokens": 256},
    return_timestamps=True
)

In [ ]:
!pip install git+https://github.com/huggingface/speechbox

In [ ]:
# Align timestamps between the diarization and Whisper
from speechbox import ASRDiarizationPipeline

pipeline = ASRDiarizationPipeline(
    asr_pipeline=asr_pipeline, diarization_pipeline=diarization_pipeline
)
# Can also be instantiated from pre-trained
#pipeline = ASRDiarizationPipeline.from_pretrained("openai/whisper-base")

In [ ]:
# Pass the audio and check the output
pipeline(sample["audio"].copy())

In [ ]:
# Define two functions to format the timestamps

# Converts a tuple of timestamps to a string, rounded to
# a set number of decimal places
def tuple_to_string(start_end_tuple, ndigits=1):
    return str((round(start_end_tuple[0], ndigits),
                round(start_end_tuple[1], ndigits)))


# Combines the speaker id, timestamp and text information onto one line,
# and splits each speaker onto their own line for ease of reading
def format_as_transcription(raw_segments):
    return "\n\n".join(
        [
            chunk["speaker"] + " " + tuple_to_string(
                chunk["timestamp"]) + chunk["text"]
            for chunk in raw_segments
        ]
    )

In [ ]:
# Re-run using the formatting
outputs = pipeline(sample["audio"].copy())

format_as_transcription(outputs)

## Hands-on exercise

### Speech translation (into French)

In [1]:
# Temporary fix to install sentencepiece
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install sentencepiece

# Restart Runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00


In [1]:
# Needed to run VitsModel and VitsTokenizer at this time
!pip install git+https://github.com/hollance/transformers.git@6900e8ba6532162a8613d2270ec2286c3f58f57b

  Cloning https://github.com/hollance/transformers.git (to revision 6900e8ba6532162a8613d2270ec2286c3f58f57b) to /tmp/pip-req-build-mytb710i
  Running command git clone --filter=blob:none --quiet https://github.com/hollance/transformers.git /tmp/pip-req-build-mytb710i
  Running command git rev-parse -q --verify 'sha^6900e8ba6532162a8613d2270ec2286c3f58f57b'
  Running command git fetch -q https://github.com/hollance/transformers.git 6900e8ba6532162a8613d2270ec2286c3f58f57b
  Running command git checkout -q 6900e8ba6532162a8613d2270ec2286c3f58f57b
  Resolved https://github.com/hollance/transformers.git to commit 6900e8ba6532162a8613d2270ec2286c3f58f57b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
#!pip install transformers
# Included in the above cell

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00


In [2]:
# Load Whisper small (244M params) for this demo
import torch
from transformers import pipeline

# Use GPU if available
device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline("automatic-speech-recognition",
                model="openai/whisper-small",
                device=device
)

In [5]:
# Load a spanish audio sample from VoxPopuli
from datasets import load_dataset

dataset = load_dataset("facebook/voxpopuli", "es",
                       split="validation", streaming=True)
sample = next(iter(dataset))

In [6]:
# (Optional) Listen to the sample
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [7]:
# Define a function to translate an audio, in French here
def translate(audio):
    outputs = pipe(audio, max_new_tokens=256,
                   generate_kwargs={"task": "transcribe", "language": "fr"})
    return outputs["text"]


# Apply to the sample
translate(sample["audio"].copy())

" M. le Président, à la différence des deux orateurs qui m'accavent de procéder, je suis d'accord avec la majorité des interviendres."

In [11]:
# Save the sample to test it
french_sample = translate(sample["audio"].copy())

In [8]:
# Compare to the source: translation is not fully accurate
sample["raw_text"]

'Señor Presidente, a diferencia de las dos oradoras que me acaban de preceder, yo estoy de acuerdo con la mayor parte de los intervinientes.'

### Text-to-speech

In [10]:
# Load the model checkpoint and tokenizer
from transformers import VitsModel, VitsTokenizer

model = VitsModel.from_pretrained("Matthijs/mms-tts-fra")
tokenizer = VitsTokenizer.from_pretrained("Matthijs/mms-tts-fra")

In [22]:
# Define function to generate the waveform output
def synthesise(text):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"]

    with torch.no_grad():
      outputs = model(input_ids)

    return outputs.audio[0]

In [23]:
# Generate the waveform output
speech_fr = synthesise(french_sample)

In [24]:
# Listen to it
Audio(speech_fr, rate=16000)

### Build a STST demo

In [25]:
# Sanity check to make sure the two models work together
import numpy as np

target_dtype = np.int16 # format expected by Gradio
max_range = np.iinfo(target_dtype).max


def speech_to_speech_translation(audio):
    translated_text = translate(audio)
    synthesised_speech = synthesise(translated_text)
    synthesised_speech = (
        synthesised_speech.numpy() * max_range).astype(target_dtype)
    return 16000, synthesised_speech

In [26]:
# Check that it gives the expected result
sampling_rate, synthesised_speech = speech_to_speech_translation(
    sample["audio"])

Audio(synthesised_speech, rate=sampling_rate)

In [27]:
# Temporary fix to install Gradio
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 10.2 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579

In [29]:
# Link notebook to the hub
from huggingface_hub import notebook_login

notebook_login()

In [31]:
# After setting space in Hugging Face, clone it here
!git clone https://huggingface.co/spaces/64FC/demo_ES_to_FR_translator

Cloning into 'demo_ES_to_FR_translator'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (4/4), 654 bytes | 654.00 KiB/s, done.


In [32]:
# Build the Gradio demo
import gradio as gr

demo = gr.Blocks()

mic_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy")
    )

file_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy")
)

with demo:
    gr.TabbedInterface(
        [mic_translate, file_translate], ["Microphone", "Audio File"])

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:950: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b7e8a74e323e2205f5.gradio.live
